In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import normalize
import numpy as np
import nengo
import tensorflow as tf
import nengo_dl

In [2]:
dataset = load_breast_cancer()
train_data = dataset.data
train_labels = dataset.target

test_data = dataset.data
test_labels = dataset.target

In [3]:
def classification_accuracy(y_true, y_pred):
    return tf.metrics.sparse_categorical_accuracy(
        y_true[:, -1], y_pred[:, -1])

with nengo.Network(seed=0) as net:
    # set some default parameters for the neurons that will make
    # the training progress more smoothly
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([100])
    net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
    net.config[nengo.Connection].synapse = None
    neuron_type = nengo.LIF(amplitude=0.01)

    # this is an optimization to improve the training speed,
    # since we won't require stateful behaviour in this example
    nengo_dl.configure_settings(stateful=False)

    inp = nengo.Node(np.zeros(30))
    
    x = nengo_dl.Layer(tf.keras.layers.Dense(units=100))(inp)
    x = nengo_dl.Layer(neuron_type)(x)

    out = nengo_dl.Layer(tf.keras.layers.Dense(units=2))(x)

    out_p = nengo.Probe(out, label="out_p")
    out_p_filt = nengo.Probe(out, synapse=0.1, label="out_p_filt")


minibatch_size = 5
sim = nengo_dl.Simulator(net, minibatch_size=minibatch_size)

train_images = train_data[:, None, :]
train_labels = train_labels[:, None, None]

n_steps = 50
test_images = np.tile(test_data[:, None, :], (1, n_steps, 1))
test_labels = np.tile(test_labels[:, None, None], (1, n_steps, 1))

Build finished in 0:00:00
Optimization finished in 0:00:00
Construction finished in 0:00:00


In [4]:
sim.compile(optimizer=tf.optimizers.RMSprop(0.001), loss={out_p: tf.losses.SparseCategoricalCrossentropy(from_logits=True)})
sim.fit(train_images, {out_p: train_labels}, epochs=100)
sim.compile(loss={out_p_filt: classification_accuracy})
print("accuracy after training:",sim.evaluate(test_images, {out_p_filt: test_labels}, verbose=0)["loss"])

Train on 565 samples
Epoch 1/100
565/565 [==============================] - 0s 383us/sample - loss: 0.2937 - out_p_loss: 0.2937
Epoch 2/100
565/565 [==============================] - 0s 313us/sample - loss: 0.2351 - out_p_loss: 0.2351
Epoch 3/100
565/565 [==============================] - 0s 319us/sample - loss: 0.2161 - out_p_loss: 0.2161
Epoch 4/100
565/565 [==============================] - 0s 390us/sample - loss: 0.2308 - out_p_loss: 0.2308
Epoch 5/100
565/565 [==============================] - 0s 324us/sample - loss: 0.2084 - out_p_loss: 0.2084
Epoch 6/100
565/565 [==============================] - 0s 272us/sample - loss: 0.2116 - out_p_loss: 0.2116
Epoch 7/100
565/565 [==============================] - 0s 470us/sample - loss: 0.2121 - out_p_loss: 0.2121
Epoch 8/100
565/565 [==============================] - 0s 417us/sample - loss: 0.2057 - out_p_loss: 0.2057
Epoch 9/100
565/565 [==============================] - 0s 417us/sample - loss: 0.2118 - out_p_loss: 0.2118
Epoch 10/100
565